In [2]:
import numpy as np
import pyvista as pv
import open3d
import modeling
from modeling import *
import scipy.ndimage
import scipy.stats
import copy
from utils import *
from matplotlib import pyplot as plt
from matplotlib import cm
#import tensorflow as tf

import random

[Warning] Since Open3D 0.15, installing Open3D via conda is deprecated. Please re-install Open3D via: `pip install open3d -U`.
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
silver = np.array([0,1,0])#[0.753, 0.753, 0.753])#{'color': [0.753, 0.753, 0.753], 'material': 
a_si = np.array([1,0,0])#[0.600, 0.460, 0.357])
gold = np.array([0,.5,1])#[212/255, 175/255, 55/255])
zr = np.array([1, .5, 0])
ox = np.array([0, 1, 1])

# Silicon + noble metal

In [115]:
filename = 'STF_Si_patterned_L768_x-0.3333333333333333_Th85.5_D5_N3029625_1661919009'
#filename = 'SCTF_Si_Ag_L256_x1.0_Th85.5_D20_N589824_1658946745'

grid, deposited, params = loadSparse('structures//' + filename + '.npz')
print(params)
m = np.max(grid, axis=0)

grid_full = matrixFromSparse(grid)

slice_point = 5

grid_si = np.where(grid_full == 1, 1, 0)[slice_point:,:,:]
grid_ag = np.where(grid_full == 2, 1, 0)[slice_point:,:,:]

# Remove largest object (the walls)
# regions, number_of_components = scipy.ndimage.label(grid_si, structure=np.ones([3,3,3]))
# inv_regions = np.where(regions < 2, 0, 1)
# grid_si = grid_si * inv_regions

grid_si = np.flip(np.argwhere(grid_si == 1), axis=1) + [0,0,slice_point]#sparseFromMatrix(grid_si)
grid_ag = np.flip(np.argwhere(grid_ag == 1), axis=1) + [0,0,slice_point]#sparseFromMatrix(grid_ag)

[{'L': 768, 'theta': 85.5, 'phi': 180.0, 'H': 128, 'D': 5, 'turns': -0.3333333333333333, 'stepper resolution': 0, 'species': [1], 'weights': array([[1., 0.],
       [0., 1.]]), 'repetition': 4194304, 'time': 4162.294365644455}]


In [116]:
print(grid_si)

SiCloud = open3d.geometry.PointCloud()
SiCloud.points = open3d.utility.Vector3dVector(grid_si)
SiCloud.paint_uniform_color(a_si)

AgCloud = open3d.geometry.PointCloud()
AgCloud.points = open3d.utility.Vector3dVector(grid_ag)
AgCloud.paint_uniform_color(a_si)

[[  1   0   5]
 [  2   0   5]
 [  3   0   5]
 ...
 [384 765  95]
 [384 766  95]
 [384 767  95]]


PointCloud with 0 points.

In [117]:
m = np.max(grid_si, axis=0)
pts = np.asarray(SiCloud.points)
colors = np.asarray(SiCloud.colors)
colors[:,:] = a_si
colors = np.column_stack([0.70*colors[:,0] * pts[:,2]/m[2]+.15, 0.85*colors[:,1] * pts[:,2]/m[2]+.15, 0.85*colors[:,2] * pts[:,2]/m[2]+.15])
SiCloud.colors = open3d.utility.Vector3dVector(colors)

# m = np.max(grid_ag, axis=0)
# n = np.min(grid_ag, axis=0)
# pts = np.asarray(AgCloud.points)
# colors = np.asarray(AgCloud.colors)
# colors[:,:] = silver
# colors = np.column_stack([0.9*colors[:,0] * (pts[:,2]-n[2])/(m[2]-n[2])+.1, 0.85*colors[:,1] * (pts[:,2]-n[2])/(m[2]-n[2])+.15, 0.8*colors[:,2] * (pts[:,2]-n[2])/(m[2]-n[2])+.2])
# AgCloud.colors = open3d.utility.Vector3dVector(colors)

open3d.visualization.draw_geometries([SiCloud, AgCloud], width=1080)

# View only middle

In [158]:
filename = 'STF_Si_Ag_L768_Th85.5_D30_N9437184_1659178176'

grid, deposited, params = loadSparse('structures//' + filename + '.npz')
print(params)
m = np.max(grid, axis=0)

grid_full = matrixFromSparse(grid)

slice_point = 15

grid_si = np.where(grid_full == 1, 1, 0)[slice_point:,:,:]
grid_ag = np.where(grid_full == 2, 1, 0)[slice_point:,:,:]

s = grid_full.shape
regions, number_of_components = scipy.ndimage.label(np.where(grid_full[slice_point:, :, :] > 0, 1, 0))#, structure=np.ones((3,3,3)))
center_structures = np.unique(regions[0, s[1]//4:s[1]*3//4, s[2]//4:s[2]*3//4])
print(center_structures)

listo = list(center_structures)
print(len(listo))

[{'L': 768, 'theta': 85.5, 'phi': 0, 'H': 72, 'D': 10, 'turns': 0, 'species': [1], 'weights': array([[ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ],
       [ 0.2       , -0.30000001,  3.        ]]), 'repetition': 8388608, 'time': 15205.85223698616}, {'L': 768, 'theta': 85.5, 'phi': 0, 'H': 96, 'D': 30, 'turns': 0, 'species': [2], 'weights': array([[ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ],
       [ 0.2       , -0.30000001,  3.        ]]), 'repetition': 1048576, 'time': 3527.082565307617}]
[   0  659  684  697  703  807  822  825  827  832  835  836  840  844
  845  847  848  849  850  851  855  856  860  861  862  863  864  865
  869  870  873  875  876  877  878  879  881  882  883  884  885  886
  888  889  890  891  892  894  895  896  898  899  901  906  907  912
  913  914  915  916  917  918  923  924  925  926  931  932  933  934
  935  936  937  943  947  948  949  950  951  959  960  961  96

In [145]:
sizes = scipy.ndimage.sum(np.where(grid_full > 0, 1, 0), regions, range(number_of_components+1))

NameError: name 'ndimage' is not defined

In [159]:
if 0 in listo:
    listo.remove(0)
grid_si_choice = np.zeros(grid_si.shape)
grid_ag_choice = np.zeros(grid_ag.shape)

upper = 100000
lower = 10000
    
for i in range(len(listo)):
    structure = listo.pop(0)#random.sample(listo, k=1)
    choice = np.where(regions == structure, 1, 0)

    '''for k in range(regions.shape[0]):
        for j in range(regions.shape[1]):
            for i in range(regions.shape[2]):
                if regions[k,j,i] in center_structures:
                    regions[k,j,i] = 1
                else:
                    regions[k,j,i] = 0'''

    #regions = np.where(regions in center_structures, 1, 0)

    grid_si_choice = grid_si * choice
    grid_ag_choice = grid_ag * choice
    
    if np.sum(grid_si_choice + grid_ag_choice) > upper or np.sum(grid_si_choice + grid_ag_choice) < lower:
        continue
    else:
        break

grid_si_choice = np.flip(np.argwhere(grid_si_choice == 1), axis=1) + [0,0,slice_point]#sparseFromMatrix(grid_si)
grid_ag_choice = np.flip(np.argwhere(grid_ag_choice == 1), axis=1) + [0,0,slice_point]#sparseFromMatrix(grid_ag)

print(grid_si_choice)
print(grid_ag_choice)

print(grid_si_choice.shape[0] + grid_ag_choice.shape[0])

[[350 155  15]
 [351 155  15]
 [353 155  15]
 ...
 [269 190  63]
 [272 190  63]
 [268 192  63]]
[[282 155  47]
 [275 152  50]
 [275 153  50]
 ...
 [253 169  75]
 [251 170  75]
 [253 170  75]]
39279


In [151]:
print(listo)

[]


In [160]:
SiCloud = open3d.geometry.PointCloud()
SiCloud.points = open3d.utility.Vector3dVector(grid_si_choice)
SiCloud.paint_uniform_color(a_si)

AgCloud = open3d.geometry.PointCloud()
AgCloud.points = open3d.utility.Vector3dVector(grid_ag_choice)
AgCloud.paint_uniform_color(a_si)

PointCloud with 6842 points.

In [ ]:
m = np.max(grid_si_choice, axis=0)
pts = np.asarray(SiCloud.points)
colors = np.asarray(SiCloud.colors)
colors[:,:] = a_si
colors = np.column_stack([0.70*colors[:,0] * pts[:,2]/m[2]+.15, 0.85*colors[:,1] * pts[:,2]/m[2]+.15, 0.85*colors[:,2] * pts[:,2]/m[2]+.15])
SiCloud.colors = open3d.utility.Vector3dVector(colors)

m = np.max(grid_ag_choice, axis=0)
n = np.min(grid_ag_choice, axis=0)
pts = np.asarray(AgCloud.points)
colors = np.asarray(AgCloud.colors)
colors[:,:] = silver
colors = np.column_stack([0.9*colors[:,0] * (pts[:,2]-n[2])/(m[2]-n[2])+.1, 0.85*colors[:,1] * (pts[:,2]-n[2])/(m[2]-n[2])+.15, 0.8*colors[:,2] * (pts[:,2]-n[2])/(m[2]-n[2])+.2])
AgCloud.colors = open3d.utility.Vector3dVector(colors)

open3d.visualization.draw_geometries([SiCloud, AgCloud], width=1080)

# ZrO2

In [190]:
filename = 'STF_ZrO2_L384_Th85.5_D5_N1048570_1663882727'

grid, deposited, params = loadSparse('structures//' + filename + '.npz')
print(params)
m = np.max(grid, axis=0)

grid_full = matrixFromSparse(grid)

slice_point = 1

grid_zr = np.where(grid_full == 2, 1, 0)[slice_point:,:,:]
grid_ox = np.where(grid_full == 3, 1, 0)[slice_point:,:,:]

grid_zr = np.flip(np.argwhere(grid_zr == 1), axis=1) + [0,0,slice_point]#sparseFromMatrix(grid_si)
grid_ox = np.flip(np.argwhere(grid_ox == 1), axis=1) + [0,0,slice_point]#sparseFromMatrix(grid_ag)

[{'L': 384, 'theta': 85.5, 'phi': 0, 'H': 112, 'D': 5, 'turns': 0, 'stepper resolution': 12800, 'species': [2, 3], 'weights': array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 5.],
       [0., 0., 5., 1.]]), 'repetition': 1048576, 'spread': [1, 1.832], 'time': 2229.751029253006}]


In [191]:
#print(grid[grid_si,:3].reshape(grid_si.shape[0], 3))

ZrCloud = open3d.geometry.PointCloud()
ZrCloud.points = open3d.utility.Vector3dVector(grid_zr)
ZrCloud.paint_uniform_color(zr)

OxCloud = open3d.geometry.PointCloud()
OxCloud.points = open3d.utility.Vector3dVector(grid_ox)
OxCloud.paint_uniform_color(ox)

PointCloud with 524189 points.

In [192]:
m = np.max(grid_zr, axis=0)
pts = np.asarray(ZrCloud.points)
colors = np.asarray(ZrCloud.colors)
colors[:,:] = zr
colors = np.column_stack([0.85*colors[:,0] * pts[:,2]/m[2]+.15, 0.85*colors[:,1] * pts[:,2]/m[2]+.15, 0.85*colors[:,2] * pts[:,2]/m[2]+.15])
ZrCloud.colors = open3d.utility.Vector3dVector(colors)

m = np.max(grid_ox, axis=0)
n = np.min(grid_ox, axis=0)
pts = np.asarray(OxCloud.points)
colors = np.asarray(OxCloud.colors)
colors[:,:] = ox
colors = np.column_stack([0.9*colors[:,0] * (pts[:,2]-n[2])/(m[2]-n[2])+.1, 0.9*colors[:,1] * (pts[:,2]-n[2])/(m[2]-n[2])+.1, 0.9*colors[:,2] * (pts[:,2]-n[2])/(m[2]-n[2])+.1])
OxCloud.colors = open3d.utility.Vector3dVector(colors)

open3d.visualization.draw_geometries([ZrCloud, OxCloud], width=1080)

# Continuous space simulations

In [45]:
filename = 'STF_Si_L32_Th85_D0.45_N32768_1673398759'

atoms, deposited, params = loadContinuum(filename + '.npz')

species = atoms[:,3]
sp_si = np.squeeze(np.argwhere(species == 1))
sp_ag = np.squeeze(np.argwhere(species == 2))

atoms_si = atoms[sp_si,:3]
atoms_ag = atoms[sp_ag,:3]

si_rad = atoms[sp_si[0], 4]
if sp_ag.shape[0] > 0:
    ag_rad = atoms[sp_ag[0], 4]
else:
    ag_rad = 0
    
print("Si radius: {}\nAg radius: {}".format(si_rad, ag_rad))

print(atoms_si.shape)
print(atoms_ag.shape)
print(np.max(atoms_ag[:,2]))

Si radius: 0.15000000596046448
Ag radius: 0
(32768, 3)
(0, 3)


ValueError: zero-size array to reduction operation maximum which has no identity

In [43]:
SiCloud = open3d.geometry.PointCloud()
SiCloud.points = open3d.utility.Vector3dVector(atoms_si)
SiCloud.paint_uniform_color(a_si)

AgCloud = open3d.geometry.PointCloud()
AgCloud.points = open3d.utility.Vector3dVector(atoms_ag)
AgCloud.paint_uniform_color(a_si)

PointCloud with 0 points.

In [15]:
m = np.max(atoms_si, axis=0)
pts = np.asarray(SiCloud.points)
colors = np.asarray(SiCloud.colors)
colors[:,:] = a_si
colors = np.column_stack([0.70*colors[:,0] * pts[:,2]/m[2]+.15, 0.85*colors[:,1] * pts[:,2]/m[2]+.15, 0.85*colors[:,2] * pts[:,2]/m[2]+.15])
SiCloud.colors = open3d.utility.Vector3dVector(colors)

if atoms_ag.shape[0] > 0:
    m = np.max(atoms_ag, axis=0)
    n = np.min(atoms_ag, axis=0)
    pts = np.asarray(AgCloud.points)
    colors = np.asarray(AgCloud.colors)
    colors[:,:] = gold
    colors = np.column_stack([0.9*colors[:,0] * (pts[:,2]-n[2])/(m[2]-n[2])+.1, 0.85*colors[:,1] * (pts[:,2]-n[2])/(m[2]-n[2])+.15, 0.8*colors[:,2] * (pts[:,2]-n[2])/(m[2]-n[2])+.2])
    AgCloud.colors = open3d.utility.Vector3dVector(colors)

open3d.visualization.draw_geometries([SiCloud, AgCloud], width=1080)

In [46]:
SiMesh = open3d.geometry.TriangleMesh()
sphere = open3d.geometry.TriangleMesh.create_sphere(radius=si_rad, resolution=8)
sphere.compute_vertex_normals()
m = np.max(atoms_si, axis=0)
for i in range(atoms_si.shape[0]):
    mesh = sphere.translate(atoms_si[i,:], relative=False)
    mesh.paint_uniform_color([0.70*a_si[0] * atoms_si[i,2]/m[2]+.3, 0.9*a_si[1] * atoms_si[i,2]/m[2]+.1, 0.9*a_si[2] * atoms_si[i,2]/m[2]+.1])
    SiMesh += mesh
#SiMesh.paint_uniform_color(a_si)


AgMesh = open3d.geometry.TriangleMesh()
if atoms_ag.shape[0] > 0:
    sphere = open3d.geometry.TriangleMesh.create_sphere(radius=ag_rad, resolution=8)
    sphere.compute_vertex_normals()
    m = np.max(atoms_ag, axis=0)
    n = np.min(atoms_ag, axis=0)
    for i in range(atoms_ag.shape[0]):
        mesh = sphere.translate(atoms_ag[i,:], relative=False)
        mesh.paint_uniform_color([0.9*silver[0] * (atoms_ag[i,2]-n[2])/(m[2]-n[2])+.1, 0.85*silver[1] * (atoms_ag[i,2]-n[2])/(m[2]-n[2])+.15, 0.8*silver[2] * (atoms_ag[i,2]-n[2])/(m[2]-n[2])+.2])
        AgMesh += mesh
    #AgMesh.paint_uniform_color(silver)

open3d.visualization.draw_geometries([SiMesh, AgMesh])